<a href="https://colab.research.google.com/github/jakartaresearch/quora-question-pairs/blob/master/notebook/R2_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive/"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive

import pandas as pd
import nltk
import logging
import time
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.manifold import TSNE
from tqdm import tqdm_notebook, tqdm
tqdm_notebook().pandas()
nltk.download('punkt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
data_path = 'drive/My Drive/Colab Notebooks/quora-question-pairs/data/clean_quora_duplicate_questions.csv'
data = pd.read_csv(data_path)

In [0]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,clean_question1,clean_question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what step step guide invest share market india,what step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what story kohinoor koh noor diamond,what would happen indian government stole kohi...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how increase speed internet connection using vpn,how internet speed increased hacking dns
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely how solve,find remainder when math 23 24 math divided 24 23
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,which one dissolve water quikly sugar salt met...,which fish would survive salt water


In [0]:
clean_q1 = data['clean_question1'].values
clean_q2 = data['clean_question2'].values

questions = []
for text in clean_q1:
    questions.append(str(text))
for text in clean_q2:
    questions.append(str(text))

In [0]:
sentences = [sent_tokenize(str(text)) for text in questions]
sentences = [sent[0].split() for sent in sentences]
print(len(sentences))

808574


In [0]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
print(cores)

2


In [0]:
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.start_time = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        self.start_time = time.time()
        
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Time execution per epoch : {}'.format(time.time() - self.start_time))

        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss-self.loss_previous_step))

        self.epoch += 1
        self.loss_previous_step = loss

In [0]:
# init word2vec class
w2v_model = Word2Vec(min_count=2, 
                     window=5, 
                     size=300,
                     sg=1, # if 1, then skipgram is used; if 0, then cbow 
                     workers=cores)

# build vovab
w2v_model.build_vocab(sentences)

# train the w2v model
w2v_model.train(sentences=sentences, 
                total_examples=w2v_model.corpus_count, 
                epochs=20,
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class

Epoch #0 start
Time execution per epoch : 32.45560169219971
Loss after epoch 0: 15752476.0
Epoch #1 start
Time execution per epoch : 32.226346254348755
Loss after epoch 1: 10272076.0
Epoch #2 start
Time execution per epoch : 32.14668083190918
Loss after epoch 2: 8965104.0
Epoch #3 start
Time execution per epoch : 32.29419136047363
Loss after epoch 3: 6156704.0
Epoch #4 start
Time execution per epoch : 31.926127195358276
Loss after epoch 4: 5948288.0
Epoch #5 start
Time execution per epoch : 31.923526525497437
Loss after epoch 5: 5776568.0
Epoch #6 start
Time execution per epoch : 31.792327165603638
Loss after epoch 6: 5607868.0
Epoch #7 start
Time execution per epoch : 31.72238302230835
Loss after epoch 7: 5487540.0
Epoch #8 start
Time execution per epoch : 31.572829961776733
Loss after epoch 8: 3447424.0
Epoch #9 start
Time execution per epoch : 31.773348569869995
Loss after epoch 9: 720632.0
Epoch #10 start
Time execution per epoch : 31.481547355651855
Loss after epoch 10: 705328.0
E

(91132872, 104806100)

In [0]:
import pickle
w2v = dict(zip(w2v_model.wv.index2word, w2v_model.wv.vectors))
pickle.dump(w2v, open("drive/My Drive/Colab Notebooks/quora-question-pairs/data/w2v_embed.pkl", "wb"))

In [0]:
# useful properties
#print(w2v_model.wv['dog']) # get the word vector of 'dog'
#print(w2v_model.wv.index2word) # get the vocabularies
#print(w2v_model.wv.vectors) # get all word vectors

In [0]:
w2v_model.wv.most_similar(['dog'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dogs', 0.5482829213142395),
 ('foamy', 0.5199366807937622),
 ('breeds', 0.5008290410041809),
 ('bile', 0.4995802342891693),
 ('neutered', 0.4985637664794922),
 ('puppy', 0.49792659282684326),
 ('leash', 0.49786514043807983),
 ('heaving', 0.4935768246650696),
 ('poops', 0.49064645171165466),
 ('doberman', 0.4875103235244751)]

In [0]:
w2v_model.save('drive/My Drive/Colab Notebooks/quora-question-pairs/data/word2vec.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
